# MACHINE LEARNING - PROBABILIDADE ADMISSAO DE ESTUDANTES

* OBJETIVO: Analisar diferentes algoritmos de machine learning em um problema de regressão com o intuito de escolher a melhor ferramenta para tratar o conjunto de dados vigente e melhorar a assertividade ao prever a probabilidade de admissão de estudantes de acordo com caracteristicas fornecidas no dataset.
* ETAPAS:
1. Carregamento e processamento do dataset
2. Aplicacao de 4 modelos de regressão linear para fins comparativos: regressão linear, lasso, ridge e elastic net.
3. Ajuste de parâmetros, otimização e aplicação dos dois melhores algoritmos de machine learning.
4. Identificação das variáveis que mais impactam nos resultados da análise dos algoritmos

IMPORTANDO BIBLIOTECAS, CARREGANDO E PROCESSANDO DATASET

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [3]:
#lendo e tratando dados
dados = pd.read_csv('Admission_Predict.csv')
dados.head(3)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72


In [4]:
dados.shape

(400, 9)

In [5]:
dados.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [6]:
dados.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [7]:
dados.drop('Serial No.', axis=1, inplace=True)

In [8]:
dados.head(2)

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76


CRIANDO VARIAVEIS PREDITORA/TARGET E BASES DE TREINO/TESTE PARA MODELOS ML DE REGRESSAO

In [9]:
#variaveis target e preditora
y = dados['Chance of Admit ']
x = dados.drop('Chance of Admit ', axis=1)

CRIANDO UMA FUNCAO PARA INSTANCIAR E APLICAR OS 4 MODELOS DE REGRESSAO QUE SERAO APLICADOS AOS DADOS (COM VALIDACAO CRIZADA)

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [137]:
def modelosReg(a,b):
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet
    kfold = KFold(n_splits=5)
    x =a 
    y =b 
    reg = LinearRegression()
    ridg = Ridge()
    lass = Lasso()
    elast = ElasticNet()
    resReg = cross_val_score(reg, x, y, cv=kfold)
    resRidg = cross_val_score(ridg, x, y, cv=kfold)
    resLass = cross_val_score(lass,x,y, cv=kfold)
    resElast = cross_val_score(elast, x, y, cv=kfold)
      
    print('Resultado Reg Linear:', resReg.mean(), 'Resultado Ridge:', resRidg.mean(),
         'Resultado Lasso:', resLass.mean(), 'Resultado Elastic:', resElast.mean())          
      
modelosReg(x,y)

Resultado Reg Linear: 0.7711794121066358 Resultado Ridge: 0.770870750124389 Resultado Lasso: 0.21208324163988027 Resultado Elastic: 0.5018924119815464


APLICANDO MELHOR DOIS MELHORES MODELO SEM VALIDACAO CRUZADA: REGRESSAO LINEAR e RIDGE

In [138]:
x_treino, y_treino, x_teste, y_teste = train_test_split(x,y,test_size=0.3, random_state=10)

In [139]:
def bestModels(aa,bb):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    x = aa
    y = bb
    reg = LinearRegression()
    ridg = Ridge()
    reg.fit(x, y)
    ridg.fit(x, y)
    resultReg = reg.score(x, y)
    resultRidg = ridg.score(x, y)
    
    print('Resultado Reg Linear:', resultReg.mean(), 'Resultado Ridge:', resultRidg.mean())    

bestModels(x, y)

Resultado Reg Linear: 0.8034713719824393 Resultado Ridge: 0.8034130263211425


FEATURE SELECTION - RFE(ELIMINACAO RECURSIVA): IDENTIFICANDO VARIAVEIS MAIS IMPACTANTES PARA O RESULTADO DOS TESTES DE ML EXECUTADOS

In [144]:
#importando bibliotecas:
from sklearn.feature_selection import RFE
import numpy as np

#instanciando modelo e aplicando o algoritmo RFE:
mReg = LinearRegression()
algoRFE = RFE(estimator=mReg, n_features_to_select=3)
fit = algoRFE.fit(x,y)

#printando resultados das features:
print('Numero de features:', fit.n_features_)
print('Features Selecionadas:', fit.support_)   #vai retornar quais variaveis foram descartadas (false)
print('Melhores Features:', fit.ranking_)  

Numero de features: 3
Features Selecionadas: [False False False False  True  True  True]
Melhores Features: [5 4 2 3 1 1 1]


In [145]:
#Visualizando lista das 3 variaveis mais relevantes para os resultados do modelo:
cols = fit.get_support(indices=True)
dados.iloc[:,cols].dtypes

LOR         float64
CGPA        float64
Research      int64
dtype: object